In [1]:
import os
import keras
import keras.backend as K
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Model

from data import read_crop_list, load_structured_sample
from eval import eval_model_one_hot
from metrics import f1, f1_loss
from models import model_stack_lstm, model_Conv1D, model_Conv1D_LSTM
from training import train, create_training_folder
from keras.utils import plot_model
from training import train, create_training_folder, create_callbacks, step_decay_exp


from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding, LSTM, Normalization, BatchNormalization, Dense, Conv1D, \
    GlobalAveragePooling1D, SpatialDropout1D, Dropout, Input, Permute, Concatenate, Flatten, Reshape

from training import train, create_training_folder


Using TensorFlow backend.


In [2]:
def load_model(file_name):
    loaded_model = keras.models.load_model(file_name)
    return loaded_model

In [3]:
from training import train, create_training_folder

tf.keras.utils.get_custom_objects()
tf.keras.utils.get_custom_objects()['f1'] = f1
tf.keras.utils.get_custom_objects()['f1_loss'] = f1_loss

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [4]:

df_crops, vocab = read_crop_list()

sample = load_structured_sample()
# sample = load_undersampled_data()
print("Using sample size: {}".format(sample.shape))

y = sample[:, 11]
X = sample[:, 3:11]
sequence_length = X.shape[-1]

# X = one_hot_encoding_X(X, vocab_size=vocab_size)
y = to_categorical(y)
# random train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
# for train_index, test_index in sss.split(X, y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# y_train = to_categorical(y_train, num_classes=vocab_size)
# y_test = to_categorical(y_test, num_classes=vocab_size)

# print(np.unique(y_train).shape)

# print("===Train/test size: {}, {}".format(len(y_train), len(y_test)))

Using sample size: (31373, 12)


In [5]:
X = X/26
X = X.astype(np.float16)

In [6]:

def lsmt_base(seq_length=8, vocab_size=27, embedding_size=20, layer_size=256):
    # embeddings = Embedding(output_dim=embedding_size, input_dim=vocab_size, input_length=sequence_length)(input)
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size,  input_length=sequence_length))
    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(units = 128, recurrent_dropout=0.3, return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(units = 128, recurrent_dropout=0.3))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(vocab_size, "relu"))
    
    return model


def conv1d_base(vocab_size=27):
    model = Sequential()
    model.add(Reshape((8, 1)))
    model.add(Conv1D(filters=128, kernel_size=2, strides=1, activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=5, strides=1, activation='relu'))
#     model.add(Conv1D(32, kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D(4))
# #     model.add(Dropout(0.5))
#     model.add(GlobalMaxPooling1D())
#     model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(vocab_size, activation='relu'))    
    return model


In [7]:
K.clear_session()
model = lsmt_base()
tag = "emb_dropout"

plot_model(model, show_shapes = True)


TypeError: 'InputLayer' object is not iterable

In [8]:
EXP_FOLDER = "batchnorm"

crop_names = df_crops["description"].values.tolist()
crop_list = df_crops["idx"].values.tolist()


training_params = {
    'loss': f1_loss,
    'optimizer': 'rmsprop',
    'metrics': [f1],
    'run_eagerly': False
}

    
epochs=200

model.compile(**training_params)

folder = create_training_folder(EXP_FOLDER, tag)

#     lr_scheduller = tf.keras.callbacks.LearningRateScheduler(step_decay_exp)
#lr_scheduller = tf.keras.callbacks.ReduceLROnPlateau(step_decay_exp)
lr_scheduller = None
callbacks = create_callbacks(folder, tensor_board=True, monitor_metric="val_f1", monitor_mode="max",
                     lr_scheduller=lr_scheduller)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


train(model, X_train, y_train, X_test, y_test, epochs=epochs, callbacks=callbacks)

# save model
model_folder = '{}/model'.format(folder)
model.save(model_folder)

loss = model.evaluate(X_test, y_test, verbose=0)
print("Loss for sequence {}: {}".format(seq,loss))

# evaluate
y_hat = model.predict(X_test)
eval_model_one_hot(folder, y_test, y_hat, crop_list, crop_names)


/home/gus/workspaces/wpy/venvs/mathor/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:97: UserWarning: The TensorBoard callback does not support gradients display when using TensorFlow 2.0. The `write_grads` argument is ignored.
  warnings.warn('The TensorBoard callback does not support '


Epoch 1/200

Epoch 00001: val_f1 improved from -inf to 0.16567, saving model to /home/gus/workspaces/wpy/IACapstone/results/batchnorm/20200809_074812_emb_dropout
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /home/gus/workspaces/wpy/IACapstone/results/batchnorm/20200809_074812_emb_dropout/assets
687/687 - 3s - loss: 0.8469 - f1: 0.1531 - val_loss: 0.8341 - val_f1: 0.1657
Epoch 2/200

Epoch 00002: val_f1 improved from 0.16567 to 0.20224, saving model to /home/gus/workspaces/wpy/IACapstone/results/batchnorm/20200809_074812_emb_dropout
INFO:tensorflow:Assets written to: /home/gus/workspaces/wpy/IACapstone/results/batchnorm/20200809_074812_emb_dropout/assets
687/687 - 3s - loss: 0.7964 - f1: 0.2033 - val_loss: 0.7973 - val_f1: 0.2022
Epoch 3/200

Epoch 00003: val_f1 improved from 0.20224 to 0.22780, saving model to /home/gus/workspaces/wpy/IACapstone/results/batchnorm/20200809_074812_emb_dropout
INFO:tensorflow:Assets wr

KeyboardInterrupt: 